In [7]:
!pip install kafka-python
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from kafka import KafkaConsumer,KafkaProducer
from time import sleep
from json import dumps 
import json
import warnings



In [9]:
producer = KafkaProducer(bootstrap_servers=['3.86.175.25'], #change ip here
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [10]:
#Spotify API

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '0a75008d9ab74718a050060ff199e32e'
client_secret = '99f47c62e43f45d7bf6a247d0ba9fc75'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [11]:
for i in range(0,1500,50):
    print(i)
    tracks = sp.search(q='year:2023', type = 'track', market = ['US','CA','UK','AU','KR','NZ','SG','IN'] , limit = 50, offset = i)

    # Creating an empty lists for track features
    artist = []
    track = []
    track_popularity = []
    artist_id = []
    track_id = []
    length = []
    album= []

    for i, j in enumerate(tracks['tracks']['items']):
   
        album.append(j['album']['name'])
        artist.append(j['artists'][0]['name'])
        artist_id.append(j['artists'][0]['id'])
        track.append(j['name'])
        track_id.append(j['id'])
        track_popularity.append(j['popularity'])
        length.append(j['duration_ms'])

   # Store in the dataframe
    df = pd.DataFrame({'Artist' : artist, 'Track' : track, 'Track_id' : track_id, 'Track_popularity' : track_popularity, 'Artist_id' : artist_id,'Album':album ,'Length' : length})

    artist_popularity = []
    genres = []
    followers = []
    artist_image = []
    for i in artist_id:
        artist = sp.artist(i)
        artist_popularity.append(artist['popularity'])
        genres.append(artist['genres'])
        followers.append(artist['followers']['total'])
        artist_image.append(artist['images'][0]['url'])
    #Assigning the above features into dataframe 'df'
    df = df.assign(Artist_Popularity = artist_popularity, Genres = genres, Followers= followers,Artist_image =artist_image)

    #Creating a empty list
    track_features =[]

    #Extracting the track features by looping into track id and creating a new dataframe called "tfeatures"
    for i in df['Track_id']:
        feature = sp.audio_features(i)
        track_features.append(feature)
        tfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
        for j in track_features:
            for k in j:
                tfeatures = tfeatures.append(k, ignore_index=True)
                # Suppress all warnings
                warnings.filterwarnings("ignore")
    
    # Merging Track and Audio Features and stored in 'df1'
    df1 = pd.merge(df, tfeatures, how = 'inner', left_on = 'Track_id', right_on = 'id')
    # Dropping the unwanted features
    df1 = df1[['Artist', 'Track', 'Track_id', 'Track_popularity', 'Artist_id','Artist_Popularity','Album','Artist_image' ,'Genres', 'Followers', 'danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
    #Capitalizing the Artist and Track
    df1['Artist'] = df1['Artist'].str.capitalize()
    df1['Track'] = df1['Track'].str.capitalize()
    # Changing the data type
    df1['key'] = df1['key'].astype(int)
    df1['mode'] = df1['mode'].astype(int)
    df1['instrumentalness'] = df1['instrumentalness'].astype(float)
    df1['duration_ms'] = df1['duration_ms'].astype(int)
    # Duration_ms in milliseconds. Convert into minutes
    df1['duration_ms'] = df1.apply(lambda x: round(x['duration_ms']/60000,2), axis=1)

    # Filter rows with non-null album_name and keep the first occurrence of each track_id
    df = df[df['Album'].notnull()].drop_duplicates(subset=['Track_id'], keep='first')

     


    for index, row in df1.iterrows():
       
        dict_spotify = row.to_dict()
        producer.send('spotify',value=dict_spotify)



0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'year:2023', 'limit': 50, 'offset': 1000, 'type': 'track', 'market': ['US', 'CA', 'UK', 'AU', 'KR', 'NZ', 'SG', 'IN']} returned 400 due to Bad request.


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/search?q=year%3A2023&limit=50&offset=1000&type=track&market=US&market=CA&market=UK&market=AU&market=KR&market=NZ&market=SG&market=IN:
 Bad request., reason: None